# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
print(len(song_files))

74


In [5]:
#song_data = []
for i in range(0,len(song_files)):
    filepath = song_files[i]
    #df = pd.read_json(filepath, lines=True)
    #print(df)
    #df = df.replace({pd.np.nan: None})
    #song_data.append(df.iloc[:,[7,8,0,9,5]].values.tolist())

#print(len(song_data))

In [6]:
print(filepath)
df = pd.read_json(filepath, lines=True)
#print(df)
df = df.replace({pd.np.nan: None})
df.head(10)


/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpoint.json


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.932,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
#song_data = df[['song_id','title','artist_id','year','duration']].values
#print(song_data[0])
song_data  = df.iloc[:,[7,8,0,9,5]].values.tolist()

song_data

[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
#for i in range(0,len(song_data)):
cur.execute(song_table_insert, song_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
#df.where(pd.notnull(df), None)
#df = df.replace({pd.np.nan: None})
artist_data = df.iloc[[0],[0,4,2,1,3]].values.tolist()
artist_data

[['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [7]:
log_files = get_files('data/log_data')
print(len(log_files))

32


In [8]:
for i in range(0,len(log_files)):
    filepath = log_files[i]
    #df = pd.read_json(filepath, lines=True)
    #print(df)
filepath

'/home/workspace/data/log_data/2018/11/.ipynb_checkpoints/2018-11-01-events-checkpoint.json'

In [9]:
df = pd.read_json(filepath, lines=True)
#print(df)
df = df.replace({pd.np.nan: None})
df.head(3)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,None,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,None,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.308,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [10]:
df1 = df[df['page'] == 'NextSong']
df1.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.308,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.039,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.188,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.421,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.267,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [15]:
t = pd.to_datetime(df1['ts'],unit='ms')
t

2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = {'start_time': t.dt.time,'hour':t.dt.hour,'day':t.dt.day,'week':t.dt.week,'month':t.dt.month,
             'year':t.dt.year,'weekday':t.dt.weekday}
print(time_data)
column_labels = ['start_time','hour','day','week','month','year','weekday']

{'start_time': 2     21:01:46.796000
4     21:05:52.796000
5     21:08:16.796000
6     21:11:13.796000
7     21:17:33.796000
8     21:24:53.796000
9     21:28:54.796000
10    21:42:00.796000
12    21:52:05.796000
13    21:55:25.796000
14    22:23:14.796000
Name: ts, dtype: object, 'hour': 2     21
4     21
5     21
6     21
7     21
8     21
9     21
10    21
12    21
13    21
14    22
Name: ts, dtype: int64, 'day': 2     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
12    1
13    1
14    1
Name: ts, dtype: int64, 'week': 2     44
4     44
5     44
6     44
7     44
8     44
9     44
10    44
12    44
13    44
14    44
Name: ts, dtype: int64, 'month': 2     11
4     11
5     11
6     11
7     11
8     11
9     11
10    11
12    11
13    11
14    11
Name: ts, dtype: int64, 'year': 2     2018
4     2018
5     2018
6     2018
7     2018
8     2018
9     2018
10    2018
12    2018
13    2018
14    2018
Name: ts, dtype: int64, 'weekday': 2     3
4     3
5     3
6     3
7     3
8

In [17]:
time_df = pd.DataFrame(time_data ,columns=column_labels)
time_df

,start_time,hour,day,week,month,year,weekday
2,21:01:46.796000,21,1,44,11,2018,3
4,21:05:52.796000,21,1,44,11,2018,3
5,21:08:16.796000,21,1,44,11,2018,3
6,21:11:13.796000,21,1,44,11,2018,3
7,21:17:33.796000,21,1,44,11,2018,3
8,21:24:53.796000,21,1,44,11,2018,3
9,21:28:54.796000,21,1,44,11,2018,3
10,21:42:00.796000,21,1,44,11,2018,3
12,21:52:05.796000,21,1,44,11,2018,3
13,21:55:25.796000,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df.iloc[:,[17,2,5,3,7]]
user_df

,userId,firstName,lastName,gender,gender
0,39,Walter,Frye,M,M
1,8,Kaylee,Summers,F,F
2,8,Kaylee,Summers,F,F
3,8,Kaylee,Summers,F,F
4,8,Kaylee,Summers,F,F
5,8,Kaylee,Summers,F,F
6,8,Kaylee,Summers,F,F
7,8,Kaylee,Summers,F,F
8,8,Kaylee,Summers,F,F
9,8,Kaylee,Summers,F,F


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    try:
        cur.execute(user_table_insert, row)
    except psycopg2.IntegrityError as e:
        conn.rollback()
        print('insert failure:',e)
        continue
    else:
        conn.commit()
        

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(8) already exists.

insert failure: duplicate key value viol

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
for index, row in df1.iterrows():
    print(row.song, row.artist, row.length)
    print('rows',row.userId,row.level,row.sessionId,row.location,row.userAgent)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print("hello",results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(index)
    #startts = time_df.iloc[index, 0]
    #startts = pd.to_datetime(df1.iloc[index, 15],unit='ms')
    startts  = pd.to_datetime(row.ts, unit='ms')
    print('startts: ',startts)
    # insert songplay record
    #songplay_data = (DEFAULT, time_df.iloc[i, 0],df1.iloc[i,[17,7]],songid,artistid,df1.iloc[i,[12,8,16]])
    songplay_data = [startts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent]
    print('songplaydata',songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

You Gotta Be Des'ree 246.30812
rows 8 free 139 Phoenix-Mesa-Scottsdale, AZ "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"
hello None
2
startts:  2018-11-01 21:01:46.796000
songplaydata [Timestamp('2018-11-01 21:01:46.796000'), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
Flat 55 Mr Oizo 144.03873
rows 8 free 139 Phoenix-Mesa-Scottsdale, AZ "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"
hello None
4
startts:  2018-11-01 21:05:52.796000
songplaydata [Timestamp('2018-11-01 21:05:52.796000'), 8, 'free', None, None, 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"']
Quem Quiser Encontrar O Amor Tamba Trio 177.18812
rows 8 free 139 Phoenix

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.